<a href="https://colab.research.google.com/github/ggalarza1/stock_financials/blob/main/Stock_financials_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stock Financial Data
## How to put an Income statement in a table

In [5]:
from rich.table import Table
from rich.console import Console
import pandas as pd
import yfinance as yf
import textwrap

 # Fetch income statement data
income_statement = yf.Ticker("aapl").quarterly_financials

 # Convert to DataFrame and fill NaN with 0
income_df = income_statement.fillna(0)

 # Format numbers with commas
for column in income_df.select_dtypes(include=['number']):
     income_df[column] = income_df[column].apply('{:,}'.format)

#Changing the order of items. Placing Operating Income first.
#operating_income_index = income_df.index[income_df.index == 'Operating Income'][0]

# Changing the new order to follow Operating Income first.
#new_order = [operating_income_index] + list(income_df.index.difference([operating_income_index]))
#reordered_income_df = income_df.reindex(new_order)

# Filter out the unwanted column --> Getting rid of 6/30/2023 and 9/30/2023
#filtered_columns = [col for col in reordered_income_df.columns if col.strftime("%m/%d/%Y") not in ["06/30/2023", "09/30/2023"]]
#filtered_df = reordered_income_df[filtered_columns]

filtered_columns = [col for col in income_df.columns if col.strftime("%m/%d/%Y") not in ["06/30/2023", "09/30/2023"]]
filtered_df = income_df[filtered_columns]

# Create the table
table = Table(title="[bold]Income Statement (Millions)[bold]", show_lines=True, title_style="bold size=18")

# Add columns with custom formatting for "Metric"
table.add_column("Financials", justify="left", style="black", no_wrap=False, width=30)  # Set no_wrap=False
for date in filtered_df.columns:
    #Format date as day/month/year
    formatted_date = date.strftime("%m/%d/%Y")
    table.add_column(formatted_date, style="magenta", width=20)  # Convert Timestamp to string


# Iterate through rows of the DataFrame
for row in filtered_df.itertuples():
    # Wrap "Metric" text if longer than 30 characters
    wrapped_metric = textwrap.fill(row.Index, width=30)  # Wrap text to 30 characters

    # Prepare row data with wrapped metric
    row_data = [wrapped_metric]
    for date in filtered_df.columns:
        value = filtered_df.loc[row.Index, date]
        # Check if the current metric is one of the exceptions
        if row.Index in ["Interest Expense","Interest Income", "Interest Expense Non Operating", "Interest Income Non Operating","Tax Rate For Cals", "Basic EPS", "Diluted EPS", "Tax Effect Of Unusual Items","Net Non Operating Interest Income Expense", "Net Interest Income",]:
            row_data.append(value)  # Keep the original value
        else:
          try:
            # Convert value to numeric and divide by 1 million
            value = float(value.replace(",", "")) / 1_000_000
            # Format value with commas
            row_data.append("{:,.0f}".format(value))
          except ValueError:
              row_data.append(str(filtered_df.loc[row.Index, date]))

    # Add the row to the table
    table.add_row(*row_data)

# Print the table using Rich
console = Console()
console.print(table)

<ipython-input-5-354d61166248>:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  income_df = income_statement.fillna(0)


Income Statement (Millions)                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Financials               ┃ 12/31/2024     ┃ 09/30/2024     ┃ 06/30/2024      ┃ 03/31/2024     ┃ 12/31/2023      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Tax Effect Of Unusual    │ 0.0            │ 0.0            │ 0.0             │ 0.0            │ 0.0             │
│ Items                    │                │                │                 │                │                 │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Tax Rate For Calcs       │ 0              │ 0              │ 0               │ 0              │ 0               │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Normalized EBITDA        │ 45,912         │ 32,502         │ 28,202          │ 30,736         │ 43,221          │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Net Income From          │ 36,330         │ 14,736         │ 21,448          │ 23,636         │ 33,916          │
│ Continuing               │                │                │                 │                │                 │
│ Operation Net Minority   │                │                │                 │                │                 │
│ Interest                 │                │                │                 │                │                 │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Reconciled Depreciation  │ 3,080          │ 2,911          │ 2,850           │ 2,836          │ 2,848           │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Reconciled Cost Of       │ 66,025         │ 51,051         │ 46,099          │ 48,482         │ 64,720          │
│ Revenue                  │                │                │                 │                │                 │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ EBITDA                   │ 45,912         │ 32,502         │ 28,202          │ 30,736         │ 43,221          │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ EBIT                     │ 42,832         │ 29,591         │ 25,352          │ 27,900         │ 40,373          │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Net Interest Income      │ 0.0            │ 0.0            │ 0.0             │ 0.0            │ 0.0             │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Interest Expense         │ 0.0            │ 0.0            │ 0.0             │ 0.0            │ 0.0             │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Interest Income          │ 0.0            │ 0.0            │ 0.0             │ 0.0            │ 0.0             │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Normalized Income        │ 36,330         │ 14,736         │ 21,448          │ 23,636         │ 33,916          │
├──────────────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ Net Income From          │ 36,330         │ 14,736         │ 21,448          │ 23,636         │ 33,916          │
│ Continuing And           │                │                │                 │                │   

In [ ]:
using PlotlyJS, CSV, DataFrames

df = dataset(DataFrame, "gapminder")
df_canada = df[df.country .== "Canada", :]

plot(df_canada, x=:year, y=:pop, kind="bar")

## Putting stock balance sheet into a table

In [ ]:
from rich.table import Table
from rich.console import Console
import pandas as pd
import yfinance as yf
import textwrap

 # Fetch income statement data
balance_statement = yf.Ticker("aapl").balance_sheet

 # Convert to DataFrame and fill NaN with 0
balance_df = balance_statement.fillna(0)

 # Format numbers with commas
for column in balance_df.select_dtypes(include=['number']):
     balance_df[column] = balance_df[column].apply('{:,}'.format)

#Changing the order of items. Placing Operating Income first.
operating_income_index = balance_df.index[balance_df.index == 'Operating Income'][0]

# Changing the new order to follow Operating Income first.
new_order = [operating_income_index] + list(income_df.index.difference([operating_income_index]))
reordered_income_df = income_df.reindex(new_order)

# Filter out the unwanted column --> Getting rid of 6/30/2023 and 9/30/2023
filtered_columns = [col for col in reordered_income_df.columns if col.strftime("%m/%d/%Y") not in ["06/30/2023", "09/30/2023"]]
filtered_df = reordered_income_df[filtered_columns]

# Create the table
table = Table(title="[bold]Income Statement (Millions)[bold]", show_lines=True, title_style="bold size=18")

# Add columns with custom formatting for "Metric"
table.add_column("Financials", justify="left", style="black", no_wrap=False, width=30)  # Set no_wrap=False
for date in filtered_df.columns:
    #Format date as day/month/year
    formatted_date = date.strftime("%m/%d/%Y")
    table.add_column(formatted_date, style="magenta", width=20)  # Convert Timestamp to string


# Iterate through rows of the DataFrame
for row in filtered_df.itertuples():
    # Wrap "Metric" text if longer than 30 characters
    wrapped_metric = textwrap.fill(row.Index, width=30)  # Wrap text to 30 characters

    # Prepare row data with wrapped metric
    row_data = [wrapped_metric]
    for date in filtered_df.columns:
        value = filtered_df.loc[row.Index, date]
        # Check if the current metric is one of the exceptions
        if row.Index in ["Interest Expense","Interest Income", "Interest Expense Non Operating", "Interest Income Non Operating","Tax Rate For Cals", "Basic EPS", "Diluted EPS", "Tax Effect Of Unusual Items","Net Non Operating Interest Income Expense", "Net Interest Income",]:
            row_data.append(value)  # Keep the original value
        else:
          try:
            # Convert value to numeric and divide by 1 million
            value = float(value.replace(",", "")) / 1_000_000
            # Format value with commas
            row_data.append("{:,.0f}".format(value))
          except ValueError:
              row_data.append(str(filtered_df.loc[row.Index, date]))

    # Add the row to the table
    table.add_row(*row_data)

# Print the table using Rich
console = Console()
console.print(table)

In [4]:
from rich.table import Table
from rich.console import Console
import pandas as pd
import yfinance as yf
import textwrap

 # Fetch income statement data
balance_statement = yf.Ticker("aapl").quarterly_balance_sheet

 # Convert to DataFrame and fill NaN with 0
balance_df = balance_statement.fillna(0)

 # Format numbers with commas
for column in balance_df.select_dtypes(include=['number']):
     balance_df[column] = balance_df[column].apply('{:,}'.format)

# Filter out the unwanted column --> Getting rid of 6/30/2023 and 9/30/2023
filtered_columns = [col for col in balance_df.columns if col.strftime("%m/%d/%Y") not in ["09/30/2023"]]
filtered_df = balance_df[filtered_columns]

filtered_df

<ipython-input-4-3a5cd97b71e4>:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  balance_df = balance_statement.fillna(0)


,2024-09-30,2022-09-30,2021-09-30,2020-09-30
Treasury Shares Number,0.0,0.0,0.0,0.0
Ordinary Shares Number,"15,116,786,000.0","15,943,425,000.0","16,426,786,000.0",0.0
Share Issued,"15,116,786,000.0","15,943,425,000.0","16,426,786,000.0",0.0
Net Debt,"76,686,000,000.0","96,423,000,000.0","89,779,000,000.0",0.0
Total Debt,"106,629,000,000.0","132,480,000,000.0","136,522,000,000.0",0.0
...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,"65,171,000,000.0","48,304,000,000.0","62,639,000,000.0",0.0
Other Short Term Investments,"35,228,000,000.0","24,658,000,000.0","27,699,000,000.0",0.0
Cash And Cash Equivalents,"29,943,000,000.0","23,646,000,000.0","34,940,000,000.0",0.0
Cash Equivalents,"2,744,000,000.0","5,100,000,000.0","17,635,000,000.0",0.0


In [ ]:
from rich.table import Table
from rich.console import Console
import pandas as pd
import yfinance as yf
import textwrap

 # Fetch income statement data
balance_statement = yf.Ticker("aapl").

 # Convert to DataFrame and fill NaN with 0
balance_df = balance_statement.fillna(0)

 # Format numbers with commas
for column in balance_df.select_dtypes(include=['number']):
     balance_df[column] = balance_df[column].apply('{:,}'.format)

# Filter out the unwanted column --> Getting rid of 6/30/2023 and 9/30/2023
filtered_columns = [col for col in balance_df.columns if col.strftime("%m/%d/%Y") not in ["09/30/2023"]]
filtered_df = balance_df[filtered_columns]

filtered_df

In [2]:
import yfinance as yf
dividends = yf.Ticker("aapl").dividends
dividends

,Dividends
Date,
1987-05-11 00:00:00-04:00,0.000536
1987-08-10 00:00:00-04:00,0.000536
1987-11-17 00:00:00-05:00,0.000714
1988-02-12 00:00:00-05:00,0.000714
1988-05-16 00:00:00-04:00,0.000714
...,...
2023-11-10 00:00:00-05:00,0.240000
2024-02-09 00:00:00-05:00,0.240000
2024-05-10 00:00:00-04:00,0.250000


In [3]:
#Plotting Apple dividends
import matplotlib.pyplot as plt
data = dividends

#Graphing the data
plt.plot(data.index, data.values)
plt.xlabel('Year')
plt.ylabel('Dividends')
plt.title('Dividend on Apple Stock (1987 - 2024)')
plt.grid(True)
plt.show()

using PlotlyJS, CSV, DataFrames

df = dividends(DataFrame, "gapminder")
df_canada = df[df.country .== "Canada", :]

plot(df_canada, x=:year, y=:pop, kind="bar")

SyntaxError: invalid syntax (<ipython-input-3-96b55c71a9e1>, line 13)

In [ ]:
using PlotlyJS, CSV, DataFrames

df = dataset(DataFrame, "gapminder")
df_canada = df[df.country .== "Canada", :]

plot(df_canada, x=:year, y=:pop, kind="bar")